In [ ]:
pip install plotly

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# Data Cleaning

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_store = pd.read_csv('data/store.csv')

In [3]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
# Merge df_store and df_train
df = df_train.merge(df_store, how='left', left_on=df_train.Store, right_on=df_store.Store)
df.drop(['key_0', 'Store_y'], axis=1, inplace=True)
df = df.rename(columns={'Store_x':'Store'})
df.shape

(1017209, 18)

In [5]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [6]:
cols = df.columns.tolist()
cols = [col.lower() for col in cols]
df.columns = cols

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   store                      1017209 non-null  int64  
 1   dayofweek                  1017209 non-null  int64  
 2   date                       1017209 non-null  object 
 3   sales                      1017209 non-null  int64  
 4   customers                  1017209 non-null  int64  
 5   open                       1017209 non-null  int64  
 6   promo                      1017209 non-null  int64  
 7   stateholiday               1017209 non-null  object 
 8   schoolholiday              1017209 non-null  int64  
 9   storetype                  1017209 non-null  object 
 10  assortment                 1017209 non-null  object 
 11  competitiondistance        1014567 non-null  float64
 12  competitionopensincemonth  693861 non-null   float64
 13  competitiono

In [8]:
df = df.assign(
    timestamp = lambda x: pd.to_datetime(x['date']),
    year = lambda x: x['timestamp'].dt.year,
    month = lambda x: x['timestamp'].dt.month,
    day = lambda x: x['timestamp'].dt.day,
    dayofyear = lambda x: x['timestamp'].dt.dayofyear
)

In [9]:
df.head()

,store,dayofweek,date,sales,customers,open,promo,stateholiday,schoolholiday,storetype,...,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear,promointerval,timestamp,year,month,day,dayofyear
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015-07-31,2015,7,31,212
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015-07-31,2015,7,31,212
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015-07-31,2015,7,31,212
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,2009.0,0,NaN,NaN,NaN,2015-07-31,2015,7,31,212
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,2015.0,0,NaN,NaN,NaN,2015-07-31,2015,7,31,212


In [10]:
df['stateholiday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [11]:
df.stateholiday.value_counts()

0    855087
0    131072
a     20260
b      6690
c      4100
Name: stateholiday, dtype: int64

In [12]:
df['stateholiday'].replace({0:'0'}, inplace=True)

In [13]:
df.stateholiday.value_counts()

0    986159
a     20260
b      6690
c      4100
Name: stateholiday, dtype: int64

In [14]:
round(df.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
store,1017209.0,558.43,321.91,1.0,280.0,558.0,838.0,1115.0
dayofweek,1017209.0,4.00,2.00,1.0,2.0,4.0,6.0,7.0
sales,1017209.0,5773.82,3849.93,0.0,3727.0,5744.0,7856.0,41551.0
customers,1017209.0,633.15,464.41,0.0,405.0,609.0,837.0,7388.0
open,1017209.0,0.83,0.38,0.0,1.0,1.0,1.0,1.0
promo,1017209.0,0.38,0.49,0.0,0.0,0.0,1.0,1.0
schoolholiday,1017209.0,0.18,0.38,0.0,0.0,0.0,0.0,1.0
competitiondistance,1014567.0,5430.09,7715.32,20.0,710.0,2330.0,6890.0,75860.0
competitionopensincemonth,693861.0,7.22,3.21,1.0,4.0,8.0,10.0,12.0
competitionopensinceyear,693861.0,2008.69,5.99,1900.0,2006.0,2010.0,2013.0,2015.0


### Handling NaN's

In [15]:
df.isnull().sum()

store                             0
dayofweek                         0
date                              0
sales                             0
customers                         0
open                              0
promo                             0
stateholiday                      0
schoolholiday                     0
storetype                         0
assortment                        0
competitiondistance            2642
competitionopensincemonth    323348
competitionopensinceyear     323348
promo2                            0
promo2sinceweek              508031
promo2sinceyear              508031
promointerval                508031
timestamp                         0
year                              0
month                             0
day                               0
dayofyear                         0
dtype: int64

In [16]:
df_store[pd.isnull(df_store.CompetitionDistance)]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [17]:
# fill NaN with a median value
df['competitiondistance'].fillna(df['competitiondistance'].median(), inplace = True)
df['competitiondistance'].isnull().sum()

0

In [18]:
tmp = df_store[pd.isnull(df_store.CompetitionOpenSinceYear  )]
tmp[tmp.CompetitionDistance != 0].shape

(354, 10)

In [19]:
tmp = df[pd.isnull(df.promo2sinceweek)]
tmp[tmp.promo2 != 0].shape

(0, 23)

In [20]:
# replace NA's by 0
df.fillna(0, inplace = True)

In [21]:
df.isnull().sum()

store                        0
dayofweek                    0
date                         0
sales                        0
customers                    0
open                         0
promo                        0
stateholiday                 0
schoolholiday                0
storetype                    0
assortment                   0
competitiondistance          0
competitionopensincemonth    0
competitionopensinceyear     0
promo2                       0
promo2sinceweek              0
promo2sinceyear              0
promointerval                0
timestamp                    0
year                         0
month                        0
day                          0
dayofyear                    0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   store                      1017209 non-null  int64         
 1   dayofweek                  1017209 non-null  int64         
 2   date                       1017209 non-null  object        
 3   sales                      1017209 non-null  int64         
 4   customers                  1017209 non-null  int64         
 5   open                       1017209 non-null  int64         
 6   promo                      1017209 non-null  int64         
 7   stateholiday               1017209 non-null  object        
 8   schoolholiday              1017209 non-null  int64         
 9   storetype                  1017209 non-null  object        
 10  assortment                 1017209 non-null  object        
 11  competitiondistance        1017209 no

In [23]:
df_test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [24]:
df_test[df_test["Open"].isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [25]:
df_test.fillna(1, inplace=True)

In [26]:
df_test.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

### Handling Datatyp

In [27]:
df_test["Open"]= df_test["Open"].astype(int)

In [28]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             41088 non-null  int64 
 1   Store          41088 non-null  int64 
 2   DayOfWeek      41088 non-null  int64 
 3   Date           41088 non-null  object
 4   Open           41088 non-null  int64 
 5   Promo          41088 non-null  int64 
 6   StateHoliday   41088 non-null  object
 7   SchoolHoliday  41088 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 2.5+ MB


### Handling Categorical Data

In [29]:
le = LabelEncoder()

In [30]:
ass_typ = pd.get_dummies(df["assortment"], drop_first=True)
df = pd.concat([df,ass_typ], axis=1)


In [31]:
df["storetype_cat"] = le.fit_transform(df["storetype"])

In [32]:
df["stateholiday_cat"] = le.fit_transform(df["stateholiday"])

In [33]:
df =df.rename(columns={"b" : "ass_extra"})

In [34]:
df =df.rename(columns={"c" : "ass_extended"})

In [35]:
df["stateholiday_cat"].unique()

array([0, 1, 2, 3])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 27 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   store                      1017209 non-null  int64         
 1   dayofweek                  1017209 non-null  int64         
 2   date                       1017209 non-null  object        
 3   sales                      1017209 non-null  int64         
 4   customers                  1017209 non-null  int64         
 5   open                       1017209 non-null  int64         
 6   promo                      1017209 non-null  int64         
 7   stateholiday               1017209 non-null  object        
 8   schoolholiday              1017209 non-null  int64         
 9   storetype                  1017209 non-null  object        
 10  assortment                 1017209 non-null  object        
 11  competitiondistance        1017209 no

In [37]:
df.stateholiday.value_counts()

0    986159
a     20260
b      6690
c      4100
Name: stateholiday, dtype: int64

In [38]:
df.stateholiday_cat.value_counts()

0    986159
1     20260
2      6690
3      4100
Name: stateholiday_cat, dtype: int64

In [39]:
df.columns

Index(['store', 'dayofweek', 'date', 'sales', 'customers', 'open', 'promo',
       'stateholiday', 'schoolholiday', 'storetype', 'assortment',
       'competitiondistance', 'competitionopensincemonth',
       'competitionopensinceyear', 'promo2', 'promo2sinceweek',
       'promo2sinceyear', 'promointerval', 'timestamp', 'year', 'month', 'day',
       'dayofyear', 'ass_extra', 'ass_extended', 'storetype_cat',
       'stateholiday_cat'],
      dtype='object')

# Eda

## Question: Are the Promos effective?

In [ ]:
df.groupby('storetype')['sales'].describe()

In [ ]:
df.groupby('storetype')['customers', 'sales'].sum()

In [ ]:
# sales trends
sns.factorplot(data = df, x = 'month', y = "sales", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo', 
               ) 

## Finding

Storetype B has the highest sales numbers, with the largest variance. All storetypes show increased sales numbers towards christmas. 

Stores which have run a promo, show higher sales. But storetypes a,c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo.

In [ ]:
# customers trends
sns.factorplot(data = df, x = 'month', y = "customers", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

## Finding

Storetype B has the highest number of customers, with the largest variance. All storetypes show an increase of customers towards christmas. This trend is higher, if they have run a promo.

Same effect of a dip for storetypes a,c and d in customers towards easter can be also be seen here.

In [ ]:
# sale per customer trends
df['salepercustomer'] = df['sales']/df['customers']
sns.factorplot(data = df, x = 'month', y = "salepercustomer", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo', 
               ) 

## Finding

Sales per customer:
storetype b seems to be where customers only buy small items in low numbers (possible trainstation location?)
storetype d customers buy the largest quantity
a und c are very similar

In [ ]:
# weekday trends
sns.factorplot(data = df, x = 'dayofweek', y = "customers", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

## Finding

Promos are run only during the work-week, no promo on saturday/sunday.

Storetype b also open on sundays -> trainstation, fo sho
storetyp a lower number of customers on saturday, c and d increased

In [ ]:
# weekday trends
sns.factorplot(data = df, x = 'dayofweek', y = "sales", 
               col = 'storetype',
               palette = 'plasma',
               hue = 'storetype',
               row = 'promo',
               ) 

Similar trends regarding sales numbers and customers.
Highest number of sales and customers on mondays, if a promo was run

## Conclusion : Question: Are the Promos effective?"
- Promos are run only during the work-week, no promo on saturday/sunday\n",
- Storetype B has the highest number of customers, with the largest variance\n",
- Storetype B has the highest sales numbers, with the largest variance\n",
- All storetypes show increased sales numbers towards christmas\n",
- Stores which have run a promo, show higher sales. But storetypes a, c and d show a dip towards easter, if they have run a promo, which is not the case for stores without a promo."

In [ ]:
## beachte: date, (assortment, storetype und stateholiday,assortment) müssen noch raus. die in der klammer sind schon durch kategorische Variablen ersetzt

In [ ]:
sns.factorplot(data = df, x = 'month', y = "sales", 
               col = 'dayofweek', 
               palette = 'plasma',
               hue = 'storetype',
               row = 'storetype',
               ) 

This shows the sales per storetyp across all months for each weekday.

In [ ]:
# stores which are opened on Sundays
df[(df.open == 1) & (df.dayofweek == 7)]['store'].unique()

These are the stores, which are open on sundays.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,8))
sns.lineplot(x = "date", y = df.groupby(df.index).mean()["competitiondistance"] , data=df, ax=axes[0])
sns.lineplot(x = "date", y = df.groupby(df.index).mean()["competitionopensincemonth"]  , data=df, ax=axes[1])

axes[0].set_title("Date vs Competiton Distance (on average)")
axes[1].set_title("Date vs Competiton Open Since Month (on average)")
plt.show()

In [ ]:
temp_df = df.copy()
temp_df.index = temp_df.date
temp_df = temp_df.groupby(temp_df.index).mean()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,8))
sns.lineplot(x = "date", y = "competitiondistance" , data=temp_df, ax=axes[0])
sns.lineplot(x = "date", y = "competitionopensincemonth"  , data=df, ax=axes[1])

axes[0].set_title("Date vs Competiton Distance (on average)")
axes[1].set_title("Date vs Competiton Open Since Month (on average)")
plt.show()



In [ ]:
df_store.describe().T

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18,8))
sns.lineplot(x = "Store", y = "CompetitionDistance" , data=df_store, ax=axes[0])
sns.lineplot(x = "Store", y = "CompetitionOpenSinceMonth"  , data=df_store, ax=axes[1])

In [ ]:
sns.set_style("dark")
sns.histplot(data=df_store, x="CompetitionDistance")


In [ ]:
sns.countplot(x="StoreType",hue="Assortment",order=["a","b","c","d"], 
                                     data=df_store,palette="rocket_r").set_title("Number of Different Assortments per Store Type")

In [ ]:
df['assperstoretype'] = df['assortment'] + df['storetype']

In [ ]:
temp = df.groupby('assperstoretype')['sales'].agg(['mean'])
temp

In [ ]:
fig2 = px.scatter(df, x='store', y='sales', color='assperstoretype')
fig2.show()

In [ ]:
fig3 = px.scatter(df, x='store', y='sales', color='extraperstoretype')
fig3.show()

In [ ]:
df['extendedperstoretype'] = (df['ass_extended']==1).astype(str) + '_' + df['storetype']

In [ ]:
fig4 = px.scatter(df, x='store', y='sales', color='extendedperstoretype')
fig4.show()

In [ ]:
fig2.write_html("images/sales_store.html")
fig3.write_html("images/sales_extraperstoretype.html")
fig4.write_html("images/sales_extendedperstoretype.html")